# Data preprocessing - dáta **regzam**

V tomto notebooku si načítame dáta *regzam_portalvs.xml*, upravíme ich do slušnej formy pre ďalšie analýzy a následne uložíme v tejto upravenej forme do excelu/csv.

In [32]:
import pandas as pd

regzam dáta sa načítavajú z **nested xml**, elementy *Employment* a *Guarantee* majú ešte vnorené elementy, čo nám trochu sťažuje načítavanie, viď nižšie:

In [2]:
regzam_data = pd.read_xml('../data/regzam_portalvs.xml')
display(regzam_data)

,Id,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment,Guarantee
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,NaN,NaN
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,NaN,NaN
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,NaN,NaN
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,NaN,NaN
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,NaN,NaN
...,...,...,...,...,...,...,...,...
32730,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,NaN,NaN
32731,13500390262,Marcela,Harčárová,None,Mgr.,None,NaN,NaN
32732,17108478442,Jana,Juhásová,None,Mgr.,None,NaN,NaN
32733,15917135755,Miriama,Kubušová,None,Mgr.,None,NaN,NaN


- Skúšali sme pracovať s argumentom *stylesheet* pandasovej funkcie read_xml, no bolo ťažké vytvoriť stylesheet, ktorý by vedel načítať nielen vnorené elementy Empolyment a Guarantee, ale aj ich ďalšie vnorené elementy, ako napr. Workplace/University.
- Preto sme napokon zvolili iný prístup:

- Načítame xml ako string:

In [25]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import tostring

tree = ET.parse('../data/regzam_portalvs.xml')
tree = tree.getroot()  # type <class 'xml.etree.ElementTree.Element'>
t = tostring(tree, encoding='utf8', method='xml')

- Teraz treba dáta "sploštiť" (anglicky: "flatten"), aby sme sa zbavili vnorení:

- Prvý pokus bol načítať dáta do slovníka, ktorý mal ako keys názvy stĺpcov a values boli príslušné listy s hodnotami pre daný stĺpec. 
Rekurzívne sme získali "sploštený" slovník. Tento prístup mal však problém, že pre tých zamestnancov, ktorým chýbali niektoré údaje (elementy), nepridalo do príslušného "stĺpca" nič,
sĺpce mali teda rozličné dĺžky a stratili sme informáciu o tom, ktorý údaj prislúcha ktorému zamestnancovi. 
- Druhý pokus už bol úspešnejší, lebo sme dáta načítali do slovníka slovníkov, teda slovník result má ako keys identifikačné čísla zamestnancov, a ako hodnoty slovník s jednotlivými údajmi pre daného zamestnanca:

In [28]:
from collections import defaultdict
from lxml import etree

def xml_to_dict(root):
    """
    Convert the nested xml to a "flat" dictionary.
    
    Parameters:
    -----------
    root : <class 'lxml.etree._Element'> object
        The loaded nested xml.
        
    Returns:
    --------
    result : dictionary
        The flat dictionary.
    
    """
    result = defaultdict(dict)
    for child in root:
        row_id = child.attrib['Id']
        result[row_id] = xml_to_dict_helper(child)
        for key in ['GivenName', 'FamilyName', 'DegreePedagogicalAcademic', 
                    'DegreeMaster', 'DegreeDoctor', 'Employment_Workplace_University', 
                    'Employment_Type', 'Employment_Subtype', 'Employment_BeginDate', 
                    'Employment_Ratio', 'Guarantee_StudyProgram', 'Guarantee_University', 
                    'Guarantee_Faculty', 'Guarantee_FormOfStudy', 'Guarantee_LevelOfStudy', 
                    'Employment_Workplace_Faculty', 'Employment_EndDate']:
            try:
                if len(result[row_id][key]) == 1:
                    result[row_id][key] = result[row_id][key][0]
                elif len(result[row_id][key]) == 0:
                    result[row_id][key] = None
            except KeyError:
                result[row_id][key] = None
    return result

def xml_to_dict_helper(root):
    # Create a defaultdict to store the result
    result = defaultdict(list)
    # Iterate over the root element's children
    for child in root:
        # Get the child's tag name and text
        tag = child.tag
        text = child.text

        # If the child has children, recursively process them
        if child.getchildren():
            sub_dict = xml_to_dict_helper(child)
            for k, v in sub_dict.items():
                result[tag + '_' + k].extend(v)
        else:
            # If the child has no children, store its tag and text in the result
            result[tag].append(text)

    return result

In [29]:
# Parse the XML document into an <class 'lxml.etree._Element'> object
root = etree.fromstring(t)

# Convert the ElementTree object to a dictionary
result = xml_to_dict(root)

In [30]:
df = pd.DataFrame.from_dict(result).transpose().reset_index().rename(columns={"index": "Employee_ID"})
display(df)

,Employee_ID,GivenName,FamilyName,DegreePedagogicalAcademic,DegreeMaster,DegreeDoctor,Employment_Workplace_University,Employment_Type,Employment_Subtype,Employment_BeginDate,Employment_Ratio,Guarantee_StudyProgram,Guarantee_University,Guarantee_Faculty,Guarantee_FormOfStudy,Guarantee_LevelOfStudy,Employment_Workplace_Faculty,Employment_EndDate
0,18799121690,Štefan,Kočan,Doc.,JUDr.,PhD.,Vysoká škola bezpečnostného manažérstva v Koši...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2020-10-01,100,None,None,None,None,None,None,None
1,14002860472,Dušan,Korgo,DOC,JUDR,PHD,None,None,None,None,None,None,None,None,None,None,None,None
2,17922733804,Ján,Buzalka,prof.,PhDr.,CSc.,None,None,None,None,None,None,None,None,None,None,None,None
3,16290288493,Jozef,Stieranka,prof.,Ing.,PhD.,Akadémia Policajného zboru,Profesor,s titulom profesor a vysokoškolským vzdelaním ...,2013-11-01,100,"[bezpečnostnoprávna ochrana osôb a majetku, be...","[Akadémia Policajného zboru, Akadémia Policajn...","[None, None, None, None, None, None, None]","[1, 2, 1, 2, 2, 1, 2]","[1, 1, 2, 2, 2, 3, 3]",None,None
4,21401303068,Ľuboš,Wäldl,doc.,JUDr.,PhD.,Akadémia Policajného zboru,Profesor,s titulom docent a vysokoškolským vzdelaním tr...,2008-09-01,100,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32725,12110964830,Joža,Spurný,doc.,JUDr.PhDr.,Ph.D.,Vysoká škola zdravotníctva a sociálnej práce s...,Docent,s titulom docent a vysokoškolským vzdelaním tr...,2022-09-30,100,None,None,None,None,None,None,None
32726,13500390262,Marcela,Harčárová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None
32727,17108478442,Jana,Juhásová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None
32728,15917135755,Miriama,Kubušová,None,Mgr.,None,Vysoká škola zdravotníctva a sociálnej práce s...,Asistent,s vysokoškolským vzdelaním druhého stupňa,2022-09-01,50,None,None,None,None,None,None,None


Možno trochu nevýhodne sme načítali dáta o Guarantee, ale až neskôr sa ukáže, či tieto dáta potrebujeme v inej forme a teda či sa nám oplatí investovať energiu do zlepšenia.

In [31]:
df.to_csv('../data/regzam.csv')